# 1. Instance import

In [1]:
from core.instance_loader import load_cached_instance
vrp = load_cached_instance('A-n64-k9')

In [2]:
vrp

{'name': 'A-n64-k9',
 'num_vehicles': 9,
 'dimension': 64,
 'capacity': 100,
 'coordinates': array([[97., 33.],
        [57., 81.],
        [ 1., 33.],
        [55., 57.],
        [29., 37.],
        [21., 39.],
        [93., 37.],
        [ 5., 91.],
        [25., 11.],
        [47., 37.],
        [87., 25.],
        [67., 65.],
        [71., 89.],
        [67., 15.],
        [45., 79.],
        [71., 57.],
        [29.,  1.],
        [59., 79.],
        [93., 83.],
        [47., 41.],
        [51., 41.],
        [23., 93.],
        [87., 95.],
        [39., 45.],
        [45.,  7.],
        [85., 51.],
        [35., 93.],
        [47., 79.],
        [59., 91.],
        [83., 51.],
        [49., 65.],
        [21., 55.],
        [51., 21.],
        [69., 43.],
        [37., 41.],
        [37., 95.],
        [ 5., 71.],
        [37., 47.],
        [83., 73.],
        [17., 71.],
        [ 5., 71.],
        [81., 17.],
        [59., 33.],
        [63., 87.],
        [21., 77.],
        

# 2. Solution construction tests

In [3]:
from core.solution_representation import VRPSolution

## 2.1. We construct the optimal solution:

In [4]:
opt_routes = vrp['optimal_routes']
opt_cost = vrp['optimal_cost']
opt_feasible = True

In [5]:
opt_sol = VRPSolution(routes=opt_routes, instance_data=vrp)

Now the `opt_sol` variable has inside it the optimal solution - and we haven't passed in its cost or feasibility boolean, but it's supposed to be able to calculate it because it has built-in methods

The feasibility check should return `True` - this is expected given the fact that it's the optimal solution

In [6]:
opt_sol.is_feasible

True

And we can compare the calculated cost (the `cost` property of the `opt_sol` object) to the `optimal_cost` in the `vrp` dictionary

In [7]:
opt_sol.cost

np.float64(1400.8320811963101)

In [8]:
vrp['optimal_cost']

1401.0

As we can see they are equal - the `opt_sol.cost` value is given as a float and calculated from the `distance_matrix` of the instance so it is more precise than the stored cost.

## 2.2. By inspecting the routes, we also construct a suboptimal, but feasible solution

We do this by splitting a route into two. Routes are cycles beginning and ending with `DEPOT` anyway, and splitting a route into two will not violate the capacity constraint as it splits the sum into two smaller capacities, so this can be done.

In [9]:
subopt_routes = [[4, 5, 50, 31, 59, 37, 23],
  [6, 29, 25],
  [9, 54, 58, 2, 63, 47, 46, 8, 16, 60, 41],
  [10, 56, 52, 13, 61, 24, 32, 42],
  [15, 11, 57, 12, 22, 18],
  [17, 1, 27, 14, 28, 43, 38],
  [20, 19, 51, 3, 53],
  [33, 48, 62],
  [34, 55, 49, 44, 39, 40],
  [36, 7, 21, 26, 35, 30, 45]]

In [10]:
subopt_sol = VRPSolution(routes=subopt_routes, instance_data=vrp)

We check that it's feasible by accessing the `is_feasible` property:

In [11]:
subopt_sol.is_feasible

True

But the cost should increase since we have two more trips (for the additional route, from and to `DEPOT`)

In [12]:
subopt_sol.cost

np.float64(1599.909955406379)

In [13]:
if subopt_sol.cost > opt_sol.cost:
    print('Passes the test')

Passes the test


## 2.3. Finally, we construct an infeasible solution.

We do this by merging two routes from the optimal solution's routes. That way the capacity constraint will be violated for sure.

In [14]:
infeasible_routes = [[4, 5, 50, 31, 59, 37, 23],
  [6, 29, 25],
  [9, 54, 58, 2, 63, 47, 46, 8, 16, 60, 41],
  [10, 56, 52, 13, 61, 24, 32, 42],
  [15, 11, 57, 12, 22, 18],
  [17, 1, 27, 14, 28, 43, 38],
  [20, 19, 51, 3, 53, 33, 48, 62],
  [34, 55, 49, 44, 39, 40, 36, 7, 21, 26, 35, 30, 45]]

In [15]:
infeasible_sol = VRPSolution(routes=infeasible_routes, instance_data=vrp)

We expect the property `is_feasible` of the object `infeasible_sol` to hold the value `False`.

In [16]:
infeasible_sol.is_feasible

False

# 3. Metrics comparison test

In this section we shall see if the metrics truly reflect the performances of the 3 solutions in section 2.

In [17]:
from utils import metrics

In [18]:
metrics_opt = metrics.calculate_all_metrics(opt_sol, vrp)
fitness_opt = metrics.calculate_fitness(opt_sol, vrp)
metrics_subopt = metrics.calculate_all_metrics(subopt_sol, vrp)
fitness_subopt = metrics.calculate_fitness(subopt_sol, vrp)
metrics_infs = metrics.calculate_all_metrics(infeasible_sol, vrp)
fitness_infs = metrics.calculate_fitness(infeasible_sol, vrp)

Now we inspect the values. Let's take a look at `metrics_opt`.

In [19]:
metrics_opt

{'cost': np.float64(1400.8320811963101),
 'gap_from_optimal': np.float64(-0.011985639092781743),
 'dispatch_rounds': 1,
 'dispatch_rounds_increase': 0.0,
 'avg_route_utilization': np.float64(94.22222222222223),
 'is_feasible': True,
 'capacity_violations': 0,
 'num_routes': 9}

As we can see, the gap from the optimal solution is $\approx0$ (error is due to floating point precision - it's supposed to be exactly $0$); and the percentage increase of the number of dispatch rounds is $0\%$ :)

In [20]:
fitness_opt

np.float64(100.00838994736495)

We aim to maximize fitness. Since the optimal solution is known, we define the fitness function to have a maximum value of $100$ which is attained when the input is the optimal solution :)

In [21]:
metrics_subopt

{'cost': np.float64(1599.909955406379),
 'gap_from_optimal': np.float64(14.19771273421691),
 'dispatch_rounds': 2,
 'dispatch_rounds_increase': 100.0,
 'avg_route_utilization': np.float64(84.8),
 'is_feasible': True,
 'capacity_violations': 0,
 'num_routes': 10}

In [22]:
fitness_subopt

np.float64(60.061601086048164)

As we can see, for the suboptimal solution, the fitness value is $60$. This makes sense because the number of dispatch rounds doubles: the number of routes is $10$ while the number of vehicles is $9$. This results in $\left\lceil \frac{10}{9} \right\rceil = 2$ rounds instead of $1$, which is penalized. The total cost of the path is not much larger (from $1400$ to $1600$), so the solution is not twice as bad as the optimal. If a solution is exactly twice as bad as the optimal (meaning both percentage increases are $100\%$), then its fitness is $0$. This solution is suboptimal, but it is not as bad as a solution that is “twice as bad”.

In [23]:
metrics_infs

{'cost': np.float64(1342.419657608097),
 'gap_from_optimal': np.float64(-4.181323511199362),
 'dispatch_rounds': 1,
 'dispatch_rounds_increase': 0.0,
 'avg_route_utilization': np.float64(106.0),
 'is_feasible': False,
 'capacity_violations': 1,
 'num_routes': 8}

In [24]:
fitness_infs

np.float64(-97.07307354216044)

Finally, note that the fitness can also be negative. A negative fitness means the solution is worse than one that is twice as bad as the optimal.

In this sense, the fitness behaves similarly to the $R^2$ metric in regression, which can also be negative. The main difference is in interpretation: while $R^2 = 0$ corresponds to a dummy regressor that always predicts the mean, $fitness = 0$ corresponds to a solution with double the cost and double the dispatch rounds of the optimal solution.